In [1]:
import pandas as pd

В данной работе было реализовано 4 алгоритма ленивой классификации объектов, представленных бинарными признаками

### Алгоритм 1
В данном алгоритме вычисляется пересечение описания классифицируемого объекта и объектов из плюс и минус контекста. 
Затем проверяется фальсифицируемость гипотезы:
- каждый из объектов из плюс-контекста голосует за положительный результат, если его пересечение с классифицируемым объектом не вклады- вается в описание из минус-контекста.
- каждый из объектов из минус-контекста голосует за отрицательный результат, если его пересечение с классифицируемым объектом не вкладывается в описание из плюс-контекста
Фальсифицированные гипотезы не голосуют.
В итоге, решение принимается методом простого большинства.

### Алгоритм 2
В данном алгоритме, в отличие от предыдущего, для каждого примера считается его поддержка в плюс- и минус-контекстах.
В итоге, выбирается класс, соответствующий контексту с большей поддержкой.
### Алгоритм 3
В данном алгоритме, в отличие от первого, фальсифицируемые гипотезы могут учавствовать в голосовании, но только при условии, что их поддержка больше замыкания в противоположном контексте. Также здесь вводится ограничение на мощность пересечения - пересечение классифицируемого объекта и плюс- или минус-контекста должно включать сожержать на менее чем C ∗ 100% признаков.

Результы - в низу ноутбука

In [149]:
def accuracy(res):
    return float(res["positive_positive"] + res["negative_negative"]) / max(1, res["positive_positive"] + res["negative_negative"] + res["positive_negative"] + res["negative_negative"] + res["contradictory"])

def precision(res):
    return float(res["positive_positive"]) / max(1, res["positive_positive"] + res["negative_negative"])

def recall(res):
    return float(res["positive_positive"]) / max(1, res["positive_positive"] + res["positive_negative"])
    
def true_neg_pred_val(res):
    return float(res["negative_negative"]) / max(1, res["negative_negative"] + res["positive_negative"])
    
def false_pos_rate(res):
    return float(res["negative_positive"]) / max(1, res["negative_positive"] + res["negative_negative"])    

def false_neg_rate(res):
    return float(res["positive_negative"]) / max(1, res["positive_positive"] + res["positive_negative"])
    
def false_disc_rate(res):
    return float(res["negative_positive"]) / max(1, res["positive_positive"] + res["negative_positive"])

def F1_score(res):
    prec = precision(res)
    rec = recall(res)
    return 2 * prec * rec / max(1, prec + rec)

def summary(res):
    stats = {}
    stats["accuracy"] = accuracy(res)
    stats["precision"] = precision(res)
    stats["recall"] = recall(res)
    stats["f1"] = F1_score(res)
    stats["TN_Pred_Rate"] = true_neg_pred_val(res)
    stats["FP_Rate"] = false_pos_rate(res)
    stats["FN_Rate"] = false_neg_rate(res)
    stats["FDISC_Rate"] = false_disc_rate(res)
    return stats

In [3]:
test_data = pd.read_csv('lazyfca15/test1.csv')

In [4]:
test_data

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,x,x,x,x,o,o,o,x,o,positive
1,x,x,x,x,o,b,o,b,o,positive
2,x,x,x,o,o,x,o,x,o,positive
3,x,x,x,o,o,b,x,o,b,positive
4,x,x,x,b,o,b,o,o,x,positive
...,...,...,...,...,...,...,...,...,...,...
88,o,b,x,o,x,x,o,x,o,negative
89,b,x,x,o,x,x,o,o,o,negative
90,b,o,x,x,o,x,b,o,b,negative
91,b,o,x,x,o,o,x,o,x,negative


In [5]:
train_data = pd.read_csv('lazyfca15/train1.csv')

In [6]:
train_data

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,o,x,positive
2,x,x,x,x,o,o,o,b,b,positive
3,x,x,x,x,o,o,b,o,b,positive
4,x,x,x,x,o,o,b,b,o,positive
...,...,...,...,...,...,...,...,...,...,...
860,o,x,x,x,o,o,o,x,x,negative
861,o,x,o,x,x,o,x,o,x,negative
862,o,x,o,x,o,x,x,o,x,negative
863,o,x,o,o,x,x,x,o,x,negative


In [35]:
attrib_names = [
    'top-left-square',
    'top-middle-square',
    'top-right-square',
    'middle-left-square',
    'middle-middle-square',
    'middle-right-square',
    'bottom-left-square',
    'bottom-middle-square',
    'bottom-right-square',
    'class'
]

In [36]:
train_data.columns = attrib_names

In [37]:
test_data.columns = attrib_names

In [114]:
def make_intent(example: pd.Series):
    return dict(
        intent=set(f"{key}: {value}" for key, value in dict(example.drop('class')).items()),
        cls=example['class']
    )


In [137]:

def check_hypothesis_alg1(context_plus, context_minus, eintent):
#     eintent = make_intent(example)
    labels = {
        'negative': 0,
        'positive': 0
    }
    eintent = eintent['intent']
    
    for element in context_plus:
        element = element['intent']
        intersect = element & eintent
        closure = [e for e in context_minus if e['intent'].issuperset(intersect)]
        if not closure and intersect:
            labels['positive'] += 1
    
    for element in context_minus:
        element = element['intent']
        intersect = element & eintent
        closure = [e for e in context_plus if e['intent'].issuperset(intersect)]
        if not closure and intersect:
            labels['negative'] += 1
    
    if labels['negative'] > labels['positive']:
        return 'negative'
    elif labels['negative'] < labels['positive']:
        return 'positive'
    else:
        return 'contradictory'
            
        
def check_hypothesis_alg2(context_plus, context_minus, eintent):
#     eintent = make_intent(example)
    labels = {
        'negative': 0,
        'positive': 0
    }
    eintent = eintent['intent']
    
    for element in context_plus:
        element = element['intent']
        intersect = element & eintent
        support = [e for e in context_plus if e['intent'].issuperset(intersect)]
        labels['positive'] += len(support) / len(context_plus)
    
    for element in context_minus:
        element = element['intent']
        intersect = element & eintent
        support = [e for e in context_minus if e['intent'].issuperset(intersect)]
        labels['negative'] += len(support) / len(context_minus)
    
    if labels['negative'] > labels['positive']:
        return 'negative'
    elif labels['negative'] < labels['positive']:
        return 'positive'
    else:
        return 'contradictory'
    
    
def check_hypothesis_alg3(context_plus, context_minus, eintent, C):
#     eintent = make_intent(example)
    labels = {
        'negative': 0,
        'positive': 0
    }
    eintent = eintent['intent']
    
    for element in context_plus:
        element = element['intent']
        intersect = element & eintent
        
        intent_power = len(intersect) / len(element)
        support = [e for e in context_plus if e['intent'].issuperset(intersect)]
        support_idx = len(support) / len(context_plus)
        
        fals = [e for e in context_minus if e['intent'].issuperset(intersect)]
        fals_idx = len(fals) / len(context_minus)
        
        
        
        if intent_power > C and support_idx > fals_idx:
            labels['positive'] += 1
            
    
    for element in context_minus:
        element = element['intent']
        intersect = element & eintent
        
        intent_power = len(intersect) / len(element)
        support = [e for e in context_minus if e['intent'].issuperset(intersect)]
        support_idx = len(support) / len(context_minus)
        
        fals = [e for e in context_plus if e['intent'].issuperset(intersect)]
        fals_idx = len(fals) / len(context_plus)
        
        
        
        if intent_power > C and support_idx > fals_idx:
            labels['negative'] += 1
    
    if labels['negative'] > labels['positive']:
        return 'negative'
    elif labels['negative'] < labels['positive']:
        return 'positive'
    else:
        return 'contradictory'
    
            
  

In [138]:
context_minus = train_data[train_data['class'] == 'negative'].apply(make_intent, axis=1)
context_plus = train_data[train_data['class'] == 'positive'].apply(make_intent, axis=1)

In [139]:
test_intents = test_data.apply(make_intent, axis=1)

In [157]:
cv_res = {} 
cv_res['contradictory'] = 0
classes = ['negative', 'positive']
for i in classes:
    for j in classes:
        cv_res[f"{i}_{j}"] = 0
C = 1   
for test_intent in test_intents:
    
    cls = test_intent['cls']
    predict = check_hypothesis_alg1(context_plus, context_minus, test_intent)

    
    if predict == 'contradictory':
        cv_res['contradictory'] += 1
    else:
        cv_res[f"{cls}_{predict}"] += 1

In [158]:
cv_res

{'contradictory': 0,
 'negative_negative': 27,
 'negative_positive': 5,
 'positive_negative': 0,
 'positive_positive': 61}

In [159]:
summary(cv_res)

{'accuracy': 0.7652173913043478,
 'precision': 0.6931818181818182,
 'recall': 1.0,
 'f1': 0.8187919463087248,
 'TN_Pred_Rate': 1.0,
 'FP_Rate': 0.15625,
 'FN_Rate': 0.0,
 'FDISC_Rate': 0.07575757575757576}

In [160]:
cv_res = {} 
cv_res['contradictory'] = 0
classes = ['negative', 'positive']
for i in classes:
    for j in classes:
        cv_res[f"{i}_{j}"] = 0
C = 1   
for test_intent in test_intents:
    
    cls = test_intent['cls']
    predict = check_hypothesis_alg2(context_plus, context_minus, test_intent)

    
    if predict == 'contradictory':
        cv_res['contradictory'] += 1
    else:
        cv_res[f"{cls}_{predict}"] += 1

In [161]:
cv_res

{'contradictory': 0,
 'negative_negative': 0,
 'negative_positive': 32,
 'positive_negative': 0,
 'positive_positive': 61}

In [162]:
summary(cv_res)

{'accuracy': 1.0,
 'precision': 1.0,
 'recall': 1.0,
 'f1': 1.0,
 'TN_Pred_Rate': 0.0,
 'FP_Rate': 1.0,
 'FN_Rate': 0.0,
 'FDISC_Rate': 0.34408602150537637}

In [163]:
cv_res = {} 
cv_res['contradictory'] = 0
classes = ['negative', 'positive']
for i in classes:
    for j in classes:
        cv_res[f"{i}_{j}"] = 0
C = 0.1   
for test_intent in test_intents:
    
    cls = test_intent['cls']
    predict = check_hypothesis_alg3(context_plus, context_minus, test_intent, C)

    
    if predict == 'contradictory':
        cv_res['contradictory'] += 1
    else:
        cv_res[f"{cls}_{predict}"] += 1

In [164]:
cv_res

{'contradictory': 0,
 'negative_negative': 12,
 'negative_positive': 20,
 'positive_negative': 3,
 'positive_positive': 58}

In [165]:
summary(cv_res)

{'accuracy': 0.8235294117647058,
 'precision': 0.8285714285714286,
 'recall': 0.9508196721311475,
 'f1': 0.885496183206107,
 'TN_Pred_Rate': 0.8,
 'FP_Rate': 0.625,
 'FN_Rate': 0.04918032786885246,
 'FDISC_Rate': 0.2564102564102564}